# Free Field Analysis Example
Pedro Arduino - UW Computational Geomechanics Group

This example shows how to run OpenSees in DesignSafe from a jupyter notebook and how to postprocess the output results using python scripts, generate a LaTex report, and create interactive plots. 

A simple 1D free field analysis of a liquefiable soil layer is analyzed using OpenSees. An schematic of the soil profile in shown in the Figure. The soil profile consists of a 1 m dry crust, 3 m liquefiable layer, and 1 m of elastic base. The ground water table is at 2 m. An earthquake excitation is applied at the bottom of the soil column. A compliant rock is considered in the analysis. 

The results are presented in terms of:

a) Time history of acceleration at the surface and corresponding response spectra.

b) Profiles of maximum displacement, peak horizontal acceleration (PHA), maximum shear strain, and stress ratio

c) Stress strain plots for a point near the center of the liquefiable zone, and

d) Evolution of pore water pressure for a point near the center of the liquefiable zone. 

<img src = "schematic.png"  height="200" width="200" align = "center">

# Setup agave and start OpenSees job

### Setup job description

In [23]:
!pip install --user --upgrade setuptools --quiet
!pip install --user --only-binary=:all: atomicwrites==1.4.0 --quiet
!pip install --user "jsonschema<4.18.0" --quiet
!pip install git+https://github.com/DesignSafe-CI/dapi.git@tapisv3 --user --quiet

In [2]:
import os
import dapi
import uuid
from datetime import date
import json

In [3]:
# Authenticate
t = dapi.auth.init()

In [10]:
# Define inputs
cur_dir = os.getcwd()
input_uri = dapi.jobs.get_ds_path_uri(t, cur_dir)
input_filename = "N10_T3.tcl"

In [19]:
job_info = dapi.jobs.generate_job_info(t, "opensees-express", input_uri, input_filename)
job_info["maxMinutes"] = 30
job_info["execSystemLogicalQueue"] = "development"
print("\n---Job Info---\n\n" + json.dumps(job_info, indent=2))


---Job Info---

{
  "name": "opensees-express_20241001_174715",
  "appId": "opensees-express",
  "appVersion": "3.7.0",
  "execSystemId": "wma-exec-01",
  "maxMinutes": 30,
  "archiveOnAppError": true,
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/freeFieldEffectiveJupyter"
    }
  ],
  "execSystemLogicalQueue": "development",
  "nodeCount": 1,
  "coresPerNode": 1,
  "parameterSet": {
    "envVariables": [
      {
        "key": "tclScript",
        "value": "N10_T3.tcl"
      }
    ]
  }
}


### Run job

In [20]:
# Submit job
job = t.jobs.submitJob(**job_info)
jobUuid = job.uuid

In [21]:
# Monitor job status
dapi.jobs.get_status(t, jobUuid)

Waiting for job to start: 3it [00:45, 15.07s/it, Status: RUNNING]    
Monitoring job:   0%|                                                       | 0/120 [00:00<?, ?it/s]

	Status: RUNNING


Monitoring job:   6%|██▋                                            | 7/120 [01:45<28:21, 15.06s/it]

	Status: ARCHIVING


Monitoring job:   7%|███▏                                           | 8/120 [02:00<28:08, 15.08s/it]

	Status: FINISHED


'FINISHED'

In [22]:
# Get runtime summary
dapi.jobs.runtime_summary(t, jobUuid)


Runtime Summary
---------------
QUEUED  time: 00:00:00
RUNNING time: 00:01:53
TOTAL   time: 00:02:38
---------------


# Postprocess Results

### Identify job, archived location and user

In [ ]:
jobinfo = t.jobs.getJob(jobUuid=job.uuid)
jobinfo.archiveSystemDir
user = jobinfo.createdby
print(jobinfo.archiveSystemDir)

### Go to archived folder -- WIP archive files are stored in Work not accesible on OpenSees Jupyter VMs

> Fix

```json
job_description["archiveSystemId"] = "designsafe.storage.default"
job_description["archiveSystemDir"] = (
    f"{t.username}/tapis-jobs-archive/${{JobCreateDate}}/${{JobName}}-${{JobUUID}}"
)
```

In [ ]:
import os

# %cd ..
cur_dir_name = cur_dir.split("/").pop()
os.chdir(jobinfo.archiveSystemDir.replace(user, "/home/jupyter/MyData"))
if not os.path.exists(cur_dir_name):
    os.makedirs(cur_dir_name)
os.chdir(cur_dir_name)

### Import python libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Plot acceleration time history

Plot acceleration time hisotory and response spectra on log-linear scale

In [ ]:
from plotAcc import plot_acc

plot_acc()

### Plot profiles

Plot profiles of max displacement, PGA, max shear strain, stress ratio and plot stress strain near the center of liquefiable layer 

In [ ]:
from plotProfile import plot_profile

plot_profile()

### Plot excess pore water pressure

In [ ]:
from plotPorepressure import plot_porepressure

plot_porepressure()

# Generate LaTeX Report 

In [ ]:
# os.system('/usr/bin/pdflatex -interaction nonstopmode ShortReport.tex')

In [ ]:
# Before we start let us install a python package for plotting
try:
    import rst2pdf

except:
    import pip

    pip.main(["install", "--user", "rst2pdf"])
    print("********* Please restart the session ***********")

import rst2pdf

In [ ]:
import sys

!{sys.executable} -m pip install rst2pdf -q

In [ ]:
# 2024 - JupyterHub
os.system("/home/jupyter/.local/bin/rst2pdf ShortReport.rst ShortReport.pdf")
# 2022 - JupyterHub
# os.system('/opt/conda/bin/rst2pdf ShortReport.rst ShortReport.pdf')

In [ ]:
class PDF(object):
    def __init__(self, pdf, size=(200, 200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return "<iframe src={0} width={1[0]} height={1[1]}></iframe>".format(
            self.pdf, self.size
        )

    def _repr_latex_(self):
        return r"\includegraphics[width=1.0\textwidth]{{{0}}}".format(self.pdf)

In [ ]:
# pdf_fn = jobinfo.archiveSystemDir.replace(user, '/user/' + user + '/files/MyData')
pdf_fn = jobinfo.archiveSystemDir.replace("/" + user, "../../../MyData")

pdf_fn += "/"
pdf_fn += cur_dir.split("/")[-1]
pdf_fn += "/ShortReport.pdf"

In [ ]:
PDF(pdf_fn, (750, 600))

# Create Interactive Plots

### Pore water pressure

In [ ]:
from interactiveplot import createpwpplot, createDispplot

In [ ]:
createpwpplot()

### Displacement

In [ ]:
createDispplot()